<img src="logo.png">


___

<img src="DynamoDBReadPerformance2.png">

### Instalar dependencias:

In [ ]:
!pip install boto3 cython PyHamcrest

### Importar dependencias, crear una tabla, cargar algunos datos

In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import sys
import random
from time import sleep
from boto3.dynamodb.conditions import Key, Attr


dynamodb = boto3.resource('dynamodb',  region_name='us-east-1')


table = dynamodb.create_table(
        TableName= 'movies',
        KeySchema=[
            {
                'KeyType': 'HASH',
                'AttributeName': 'year'
            },
            {
                'KeyType': 'RANGE',
                'AttributeName': 'title'
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'year',
                'AttributeType': 'N'
            },
            {
                'AttributeName': 'title',
                'AttributeType': 'S'
            }           
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 2,
            'WriteCapacityUnits': 2
        }
    )
# Wait until the table exists.
table.meta.client.get_waiter('table_exists').wait(TableName='movies')
print('Table is ready, please continue as instructed.')

In [ ]:
try:
    response = dynamodb_client.create_table(
        AttributeDefinitions=[
            {
                'AttributeName': 'Artist',
                'AttributeType': 'S',
            },
            {
                'AttributeName': 'SongTitle',
                'AttributeType': 'S',
            },
        ],
        KeySchema=[
            {
                'AttributeName': 'Artist',
                'KeyType': 'HASH',
            },
            {
                'AttributeName': 'SongTitle',
                'KeyType': 'RANGE',
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5,
        },
        TableName='test',
    )
except dynamodb_client.exceptions.ResourceInUseException:
    print('Table is ready, please continue as instructed.')
    pass

### Subamos algunos datos:

In [ ]:
dynamodb = boto3.resource('dynamodb',  region_name='us-east-1')
table = dynamodb.Table('movies')

choices = ['yes', 'no']
i = 0
with open("moviedata.json") as json_file:
    with table.batch_writer() as batch:
        movies = json.load(json_file, parse_float = decimal.Decimal)
        for movie in movies:
            i = i + 1
            if i == 101:
                break
            year = int(movie['year'])
            title = movie['title']
            star = movie['actors'][0]
            rating = movie['rating']
            running_time = movie['running_time_secs']
            uploaded = random.choice(choices)

            print("Adding movie:", year, title, star, rating, running_time, uploaded)

            batch.put_item(
               Item={
                   'year': year, # agregue un +1 a esto la segunda vez.
                   'title': title,
                   'actor': star,
                   'rating': rating,
                   'running_time': running_time,
                   'uploaded' : uploaded
                }
            )


### Ahora, realicemos algunas consultas:

#### Primero, realizaremos un escaneo de nuestra mesa

In [ ]:
import pprint

dynamodb = boto3.resource('dynamodb',  region_name='us-east-2', )
table = dynamodb.Table('movies')

response = table.scan(
    ReturnConsumedCapacity='TOTAL', 
    ConsistentRead=True  # Also try with Consistent Reads
)

total_consumed_read_capacity = response['ConsumedCapacity']
print(total_consumed_read_capacity)

##### Ahora, hagamos una consulta que use la clave primaria completa

In [ ]:
import pprint

dynamodb = boto3.resource('dynamodb',  region_name='us-east-2', )
table = dynamodb.Table('movies')

response = table.query(
    ReturnConsumedCapacity='TOTAL', 
    KeyConditionExpression=Key('year').eq(2011) & Key('title').eq('Byzantium')
)


total_consumed_read_capacity = response['ConsumedCapacity']
print(total_consumed_read_capacity)


# ¡Todo terminado! ¡Buen trabajo!
<img src="buen_Trabajo.jpg">
